In [2]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.types import interrupt, Command

llm = init_chat_model("openai:gpt-5-nano")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

config = {
    "configurable": {
        "thread_id": "3",
    },
}

In [3]:
class State(MessagesState):
    custom_stuff: str

graph_builder = StateGraph(State)

In [5]:
@tool
def get_human_feedback(poem: str):
    """
    Asks the user for feedback on the poem.
    Use this before returning the final response.
    """
    feedback = interrupt(f"Here is the poem, tell me what you think\n{poem}")
    return feedback


llm_with_tools = llm.bind_tools(
    tools=[
        get_human_feedback,
    ]
)


def chatbot(state: State):
    response = llm_with_tools.invoke(
        f"""
        You are an expert in making poems.

        Use the `get_human_feedback` tool to get feedback on your poem.

        Only after you receive positive feedback you can return the final poem.

        ALWAYS ASK FOR FEEDBACK FIRST.

        Here is the conversation history:

        {state["messages"]}
    """
    )
    return {
        "messages": [response],
    }

In [ ]:
tool_node = ToolNode(
    tools=[
        get_human_feedback,
    ],
)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

# graph = graph_builder.compile()

In [1]:
graph

NameError: name 'graph' is not defined

In [7]:
result = graph.invoke(
    {
        "messages": [
            {"role": "user", "content": "Please make a poem about Python code."},
        ]
    },
    config=config,
)

In [8]:
response = Command(resume="It looks great!")

result = graph.invoke(
    response,
    config=config,
)
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Please make a poem about Python code.
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_ht5DDUaBcRL5vSfN1bV8MzXQ)
 Call ID: call_ht5DDUaBcRL5vSfN1bV8MzXQ
  Args:
    poem: Python whispers in the shell-light dawn,
Indentation serves as a patient guide.
No braces shouting; whitespace sings on,
A language where elegance and clarity reside.

Functions stretch their limbs in quiet rooms,
Lambdas slip by like fluent streams.
Lists march in loops with tidy grooms,
Dictionaries hold their chest of dreams.

Decorators cloak the brave, the bold,
Yield, then return—control in graceful arcs.
Errors become stories to be retold,
As modules bloom like code in stars.
================================= Tool Message =================================
Name: get_human_feedback

It looks great!
================================== Ai Message ===================

In [9]:
snapshot = graph.get_state(config)

snapshot.next

()